# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [3]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [9]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1/1/2023,Conflicts are represented in a cognitive space...,bioRxiv,NaN,"Yang, G., Wu, H., Li, Q., Liu, X., Fu, Z., & J...",eLife_underReview,http://guochun-yang.github.io/files/Manuscript...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/1/2022,Distinct Brain Mechanisms for Conflict Adaptat...,Journal of Cognitive Neuroscience,NaN,"Yang, G.#, Wang, K.#, Nan, W., Li, Q., Zheng, ...",JCN,http://guochun-yang.github.io/files/jocn_2022.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
2,1/1/2021,The Congruency sequence effect is modulated by...,"Journal of Experimental Psychology: Learning, ...",NaN,"Yang, G.#, Xu, H.#, Li, Z., Nan, W., Wu, H., L...",JEP-LMC,http://guochun-yang.github.io/files/JEP_LMC_20...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
3,1/1/2021,Cognitive and neural mechanisms of human gende...,Advances in Psychological Science [Chinese],NaN,"Yang, G., Wu, H., Qi, Y., & Liu, X. (2021). Co...",Gender_Review,http://guochun-yang.github.io/files/Gender_Rev...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021.0
4,1/1/2020,Dissociated Neural Mechanisms of Target and Di...,Journal of Neuroscience (Journal Club),NaN,"Li, Z., Goschl, F., & Yang, G.* (2020). Dissoc...",JN,http://guochun-yang.github.io/files/JN_2020.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.0
5,1/1/2019,Generality and specificity of cognitive contro...,Acta Physiologica Sinica [Chinese],NaN,"Yang, G., Li, Z., Wu, H., & Liu, X. (2019). Ge...",Generality_Review,http://guochun-yang.github.io/files/Generality...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
6,1/1/2017,Conflict detection and resolution rely on a co...,Neuroscience and Biobehavioral Reviews,NaN,"Li, Q.#, Yang, G. #, Li, Z., Qi, Y., Cole, M. ...",NBR,http://guochun-yang.github.io/files/NBR_2017.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
7,1/1/2017,Distinct cognitive control mechanisms as revea...,Journal of Experimental Psychology: Human Perc...,NaN,"Yang, G., Nan, W., Zheng, Y., Wu, H., Li, Q., ...",JEP-HPP,http://guochun-yang.github.io/files/JEP-HPP_20...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
8,1/1/2015,Behavioral and electrophysiological profiles r...,Advances in Computational Psychophysiology,NaN,"Yang, G., Nan, W., Li Q., & X., L. (2015). Beh...",CP,http://guochun-yang.github.io/files/CP_2015.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015.0
9,1/1/2020,Altered Reward Processing System in Internet G...,Frontiers in Psychiatry,NaN,"Raiha, S.#, Yang, G.#, Wang L., Dai W., Wu H.,...",Frontiers,http://guochun-yang.github.io/files/Frontier_2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020.0


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [10]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [12]:
item

pub_date                                                1/1/2020
title          Better regulatory performance without greater ...
venue                     Personality and Individual Differences
excerpt                                                      NaN
citation       Ouyang, Y.*, Yang, G.*, Zhu, Y., Tan, Q., & Li...
url_slug                                            Pearsonality
paper_url      http://guochun-yang.github.io/files/Personalit...
Unnamed: 7                                                   NaN
Unnamed: 8                                                   NaN
Unnamed: 9                                                   NaN
Unnamed: 10                                                  NaN
Unnamed: 11                                                  NaN
Unnamed: 12                                                  NaN
Unnamed: 13                                                  NaN
Unnamed: 14                                               2020.0
Name: 10, dtype: object

In [13]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).